### Questions

- $\gamma$ 確認(v)
- 資料集跟 paper 少部份不一樣(v)
- P1 解不出來...

### Solutions

- Check outlier(min, max)
- objective + soft margin(slack)
- non-linear kernel -> Gaussian kernel(range of $\gamma$)

In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

### Data helper

In [2]:
import os
import pandas as pd
from ucimlrepo import fetch_ucirepo 

class DataHolder():
    def __init__(self) -> None:

        self.files_id = {
            "careval": 19,
            "wisconsin": 17,
            "votes": 105,
        }

    def get(self, name):

        if name == "nursery":
            df = self.fetch_dataset(name)
#             df = df.sample(frac=1)
            X = df.loc[:, df.columns[:-1]]
            X = self.encode(X)
            y = df[df.columns[-1]].to_frame('class')
            y = self.multiclass_preprocessing(y)

        elif name == "australian":
            df = self.fetch_dataset(name)
            X = df.loc[:, df.columns[:-1]]
            X['A1'] = X['A1'].replace({0: 'a', 1: 'b'})
            X['A4'] = X['A4'].replace({1: 'p', 2: 'g', 3: 'gg'})
            X['A5'] = X['A5'].replace({1: 'ff', 2: 'd', 3: 'i', 4: 'k', 5: 'j', 6: 'aa', 7: 'm', 8: 'c', 
                                        9: 'w', 10: 'e', 11: 'q', 12: 'r', 13: 'cc', 14: 'x'})
            X['A6'] = X['A6'].replace({1: 'ff', 2: 'dd', 3: 'j', 4: 'bb', 5: 'v', 6: 'n', 7: 'o', 8: 'h',
                                        9: 'z'})
            X['A8'] = X['A8'].replace({0: 'f', 1: 't'})
            X['A9'] = X['A9'].replace({0: 'f', 1: 't'})
            X['A11'] = X['A11'].replace({0: '0', 1: 't'})
            X['A12'] = X['A12'].replace({1: 's', 2: 'g', 3: 'p'})
            X = self.encode(X)
            y = df[df.columns[-1]].to_frame('class')
            
        elif name == "gastrointestinal":
            df = self.fetch_dataset(name)
            X = df.iloc[:, 3:]
            t1 = df.iloc[:, 0].to_frame('class')
            t2 = df.iloc[:, 1].to_frame('class')
            t3 = df.iloc[:, 2].to_frame('class')
            indices_with_WL = t3[t3['class'] == '1'].index
            indices_with_NBI = t3[t3['class'] == '2'].index
            result_t2 = t2.loc[indices_with_WL]
            X = X.loc[indices_with_WL]

            y = self.multiclass_preprocessing(result_t2)
            y = self.encode(y)
            
        elif name == "votes":

            df = self.fetch_dataset(name)
            X = df.iloc[:, 1:]
            y = df.iloc[:, 0]
            X = self.encode(X)
            y = self.encode(y)

        else:
                    
            df = self.fetch_dataset(name)
            
            X = df.data.features 
            y = df.data.targets 
            
            df = pd.concat([X,y],axis=1)
            df = df.sample(frac=1) # shuffle
            
            if name == 'careval':
                X = self.encode(X)
                y = self.multiclass_preprocessing(y)

            y = self.encode(y)

        return X, y
        
    def fetch_dataset(self, name):

        prefix = 'data'+os.sep

        if name == 'nursery':

            file_path = prefix+name+os.sep+name+'.data'
            df = pd.read_csv(file_path, header=None) 
            attr = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'class']
            df.columns = attr

        elif name == 'votes':

            file_path = prefix+name+os.sep+'house-votes-84.data'
            df = pd.read_csv(file_path, delimiter=',', header=None) 
            class_name = ['class']
            features_name = [f'A{i+1}' for i in range(len(df.columns)-1)]
            column_names = class_name+features_name
            df.columns = column_names

        elif name == 'australian':

            file_path = prefix+name+os.sep+'australian.dat'
            df = pd.read_csv(file_path, delimiter=' ', header=None) 
            attr = [f'A{i+1}' for i in range(15)]
            df.columns = attr
            
        elif name == 'gastrointestinal':

            file_path = prefix+name+os.sep+'data.txt'
            df = pd.read_csv(file_path, header=None)
            list_of_rows = df.values.tolist()
            raw_features = list_of_rows[3:]
            labels_name = [f'label{i+1}' for i in range(3)]
            features_name = [f'A{i+1}' for i in range(698)]
            attr = labels_name + features_name
            df = pd.DataFrame(list_of_rows).transpose()
            df.columns = attr
            
        else:

            df = fetch_ucirepo(id=self.files_id[name]) 

        return df
    
    def impute(self, df):
        columns_with_missing_values = df.data.features.columns[df.data.features.isna().any()].tolist()

        # imputation with mode
        df.data.features.loc[:, columns_with_missing_values] = \
            df.data.features[columns_with_missing_values].fillna(df.data.features[columns_with_missing_values].mode().iloc[0])
        return df
    
    def encode(self, df):
        df = pd.get_dummies(df, drop_first=True)
        df = df.astype(int)
        return df
    
    def multiclass_preprocessing(self, y):
        
        majority_class = y['class'].mode().iloc[0]
        y_copy = y.copy()
        y_copy['binary_label'] = y_copy['class'].apply(lambda x: 1 if x == majority_class else 0)
        y_copy = y_copy.drop('class', axis=1)
        return y_copy
    
    def show_details(self, X, y):
        nums_of_row = X.shape[0]
        nums_of_feature = X.shape[1]
        positive_count = y.sum()
        print(f'nums_of_row:{nums_of_row}')
        print(f'nums_of_feature:{nums_of_feature}')
        if (positive_count.values < (nums_of_row - positive_count.values)):
            positive_count == nums_of_row - positive_count.values
        print(f'positive_count:{positive_count.values}')

if __name__ == '__main__':
    
    dh = DataHolder()
    X, y = dh.get('gastrointestinal') # not yet 我通靈不到他怎麼算的
    X, y = dh.get('nursery') # done
#     X, y = dh.get('australian') # done -> positive count 307 vs. 383
    X, y = dh.get('careval') # done 
#     X, y = dh.get('wisconsin') # done -> positive count 212 vs. 357
#     X, y = dh.get('votes') # done -> positive count 168 vs. 267 
    
    # min max
    scaler = MinMaxScaler()
    X[X.columns] = scaler.fit_transform(X[X.columns])
    
    dh.show_details(X,y)
    X = X.replace(0, -1)
    y = y.replace(0, -1)
    
    
   
    
    
    

nums_of_row:1728
nums_of_feature:15
positive_count:[1210]


In [3]:
X = X[:500]
y = y[:500]

## P1

In [4]:
def P1(X:pd.DataFrame, y:np.ndarray, c:list = None, lambda_:list = [0.5, 0.5], M2:int = 100, M3:int = 100):
    """ The cost-sensitive FS procedure """
    """ The first constraint causes infeasible """
    """ Model is feasible if zeta is continuous """
    
    # set up
    N = X.shape[1]
    size = X.shape[0]   
    x = np.array([X.iloc[i, :] for i in range(size)])
    y = np.array(y).flatten()

    if c == None:
        c = [1 for _ in range(N)]

    # create model
    model = Model("P1")
    
    # decision variables
    w = [model.addVar(vtype=GRB.CONTINUOUS) for _ in range(N)]
    beta = model.addVar(vtype=GRB.CONTINUOUS)
    zeta = [model.addVar(lb=0, ub=1, vtype=GRB.BINARY) for _ in range(size)]
    z = [model.addVar(vtype=GRB.BINARY) for _ in range(N)]
    
    
    # constraints 
    model.addConstrs(
        y[i] * (np.dot(np.array(w), x[i]) + beta) >= 1 - M2 * (1 - zeta[i]) for i in range(size)
    )
    model.addConstr(
        quicksum(zeta[i] * (1 - y[i]) for i in range(size)) >= lambda_[0] * quicksum(1 - y[i] for i in range(size))
    )
    model.addConstr(
        quicksum(zeta[i] * (1 + y[i]) for i in range(size)) >= lambda_[1] * quicksum(1 + y[i] for i in range(size))
    )
    model.addConstrs(
        w[k] <= M3 * z[k] for k in range(N)
    )
    model.addConstrs(
        w[k] >= -M3 * z[k] for k in range(N)
    )
    
    
    # objective function
    model.setObjective(quicksum(c[k] * z[k] for k in range(N)), GRB.MINIMIZE)
    
    # optimization
    model.optimize()
    
    # result
    result = {
        "w": [i.x for i in w],
        "beta": beta.x,
        "z": [i.x for i in z],
        "zeta": [i.x for i in zeta]
    }
    
    print(f'obj - {model.ObjVal}')
    
    return result

In [ ]:
res = P1(X,y)["z"]
choose = [i for i in range(len(res)) if res[i] == 1]
choose

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-24
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 532 rows, 531 columns and 9060 nonzeros
Model fingerprint: 0x9fe10d7e
Variable types: 16 continuous, 515 integer (515 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 16 rows and 2 columns
Presolve time: 0.01s
Presolved: 516 rows, 529 columns, 8528 nonzeros
Variable types: 15 continuous, 514 integer (514 binary)

Root relaxation: objective 0.000000e+00, 183 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

 370360 19281 infeasible  139         2.00000    1.00000  50.0%  42.2  367s
 375086 20607     cutoff  195         2.00000    1.00000  50.0%  42.2  371s
 378252 19999 infeasible  150         2.00000    1.00000  50.0%  42.5  376s
 381852 20033     cutoff  144         2.00000    1.00000  50.0%  42.7  380s
 385991 20158     cutoff   72         2.00000    1.00000  50.0%  42.9  385s
 389815 20126 infeasible  104         2.00000    1.00000  50.0%  43.0  390s
 391648 19851     cutoff  110         2.00000    1.00000  50.0%  43.3  395s
 395225 20319 infeasible  159         2.00000    1.00000  50.0%  43.7  403s
 398968 20066 infeasible   62         2.00000    1.00000  50.0%  43.7  407s
 400352 19882 infeasible   58         2.00000    1.00000  50.0%  44.0  412s
 401500 19904 infeasible  175         2.00000    1.00000  50.0%  44.1  415s
 404797 19375     cutoff  137         2.00000    1.00000  50.0%  44.5  422s
 406009 19497    1.00000   64  135    2.00000    1.00000  50.0%  44.5  426s
 410301 1942

In [ ]:
result = P1(X,y)

In [ ]:
result["zeta"].count(1), len(result["zeta"])

In [ ]:
result["beta"]

### 其實 P2, P3 可以只寫成一個，但我不確定會不會比較好

## P2

In [ ]:
def P2(X:pd.DataFrame, y, zk:list=None, C:int=100, lambda_:list=[0.5,0.5], M1:int=100):
    """ Cost-sensitive sparse SVMs - linear """
    # set up
    N = X.shape[1]
    size = X.shape[0]   
    x = np.array([X.iloc[i, :] for i in range(size)])
    y = np.array(y).flatten()
    
    
    
    z = [0 for _ in range(N)]
    for i in zk:
        z[i] = 1
        
    # create model 
    model = Model("P2")
    
    # decision variables
    w = [model.addVar(vtype=GRB.CONTINUOUS) for _ in range(N)]
    beta = model.addVar(vtype=GRB.CONTINUOUS)
    xi = [model.addVar(lb=0,vtype=GRB.CONTINUOUS) for _ in range(size)]
    zeta = [model.addVar(vtype=GRB.BINARY) for _ in range(size)]
    
    # constraints
    model.addConstrs(
        y[i] * (quicksum(w[j] * z[j] * x[i][j] for j in range(N)) + beta) >= 1 - xi[i] for i in range(size)
    )
    model.addConstrs(
        xi[i] <= M1 * (1 - zeta[i]) for i in range(size)
    )
    model.addConstr(
        quicksum(zeta[i] * (1 - y[i]) for i in range(size)) >= lambda_[0] * quicksum(1 - y[i] for i in range(size))
    )
    model.addConstr(
        quicksum(zeta[i] * (1 + y[i]) for i in range(size)) >= lambda_[1] * quicksum(1 + y[i] for i in range(size))
    )
    
    # objective function
    model.setObjective(quicksum(w[j] ** 2 * z[j] for j in range(N)) + C * quicksum(xi[i] for i in range(size)), GRB.MINIMIZE)
    
    # optimization
    model.optimize()
    
    
    # result
    result = {
        "w": [i.x for i in w],
        "beta": beta.x,
        "xi": [i.x for i in xi]
    }
    
    return result

In [ ]:
res = P2(X,y, choose)

In [ ]:
res["w"]

## P3

In [ ]:
def K_(z:list):
    N = len(z)
    def func(x,x_prime, gamma=0.5):
        """ gamma need to be tuned """
#         global gamma
        return np.exp(-gamma * sum(z[k] * (x[k] - x_prime[k]) ** 2 for k in range(N)))
    return func

In [ ]:
def P3(X:pd.DataFrame, y, zk:list=None, C:int=5, lambda_:list=[0.1, 0.1], M1:int=100):
    # set up
    N = X.shape[1]
    size = X.shape[0]   
    x = np.array([X.iloc[i, :] for i in range(size)])
    y = np.array(y).flatten()

    z = [0 for _ in range(N)]
    for i in zk:
        z[i] = 1
    
    global K_
    K = K_(z)
    
    # create model
    model = Model("P3")
    
    # decision variables
    alpha = [model.addVar(lb=0, ub=C/2, vtype=GRB.CONTINUOUS) for _ in range(size)]
    xi = [model.addVar(lb=0, vtype=GRB.CONTINUOUS) for _ in range(size)]
    zeta = [model.addVar(vtype=GRB.BINARY) for _ in range(size)]
    beta = model.addVar(vtype=GRB.CONTINUOUS)
    
    # constraints
    model.addConstrs(
        (y[i] * (quicksum(alpha[j] * y[j] * K(x[i], x[j]) for j in range(N)) + beta) >= 1 - xi[i]) for i in range(size)
    )
    model.addConstrs(
        (xi[i] <= M1 * (1 - zeta[i])) for i in range(size)
    )
    model.addConstr(
        quicksum(alpha[i] * y[i] for i in range(size)) == 0
    )
    model.addConstr(
        quicksum(zeta[i] * (1 - y[i]) for i in range(size)) >= lambda_[0] * (quicksum(1 - y[i] for i in range(size)))
    )
    model.addConstr(
        quicksum(zeta[i] * (1 + y[i]) for i in range(size)) >= lambda_[1] * (quicksum(1 + y[i] for i in range(size)))
    )
    
    # optimization
    model.optimize()
    
    # result
    result = {
        "alpha": [i.x for i in alpha],
        "xi": [i.x for i in xi],
        "zeta": [i.x for i in zeta],
        "beta": beta.x
    }
    
    return result

In [ ]:
P3(X, y, choose)